In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import guidance

guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)

import re
def parse_best(prosandcons, options):
    best = int(re.findall(r'Best=(\d+)', prosandcons)[0])
    return options[best]

create_plan = guidance('''<|im_start|>system
You are a helpful assistant.
<|im_end|>
<|im_start|>user
I want to {{goal}}.

{{~! generate potential options ~}}
{{#block hidden=True}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
<|im_end|>
<|im_start|>assistant
{{generate 'options' n=5 temperature=1.0 max_tokens=500}}
<|im_end|>
{{/block}}

{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
<|im_end|>
<|im_start|>assistant
{{generate 'prosandcons' temperature=0.0 max_tokens=500}}
<|im_end|>
{{/block}}

{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
<|im_end|>
<|im_start|>assistant
{{generate 'plan' max_tokens=500}}
<|im_end|>''')

out = create_plan(goal='Make a lot of money', parse_best=parse_best)
out

In [2]:
import guidance

In [3]:
guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)

In [8]:
system = '''<|im_start|>system
You are a helpful assistant<|im_end|>'''
user = '''<|im_start|>user'''
assistant = '''<|im_start|>assistant'''
end = '''<|im_end|>'''

In [37]:
eval_idea = guidance('''{{system}}
{{user}}
I want to {{goal}}.
Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.{{end}}
{{assistant}}
{{generate 'prosandcons' temperature=0.0 max_tokens=500}}''')

In [38]:
eval_idea(system=system, user=user, assistant=assistant,
end=end, goal='get a job', options=['go to college', 'send resumes out'])

{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to get a job.\nCan you please comment on the pros and cons of each of the following options, and then pick the best option?\n---\nOption 0: go to college\nOption 1: send resumes out\nOption 0: go to college\nOption 1: send resumes out\n---\nPlease discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.'}]}


<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to get a job.
Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---
Option 0: go to college
Option 1: send resumes out
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.<|im_end|>
<|im_start|>assistant
Option 0: Go to college
Pros: Gain valuable knowledge and skills, increase job opportunities.
Cons: Expensive and time-consuming.

Option 1: Send resumes out
Pros: Immediate job search, potential for quicker employment.
Cons: Limited job options without higher education.

Best=0, as going to college provides long-term benefits and increased job opportunities.

In [76]:
def eval(options, goal):
    return eval_idea(system=system, user=user, assistant=assistant, end=end, goal=goal, options=options)

def wat(chosen):
    print(type(chosen))

import re
def parse_best(options, best):
    options = options['options']
    best = int(re.findall(r'Best=(\d+)', best['prosandcons'])[0])
    return options[best]
parse_best(options, best)
goal = 'Make a lot of money'
gen_options = guidance('''{{system}}
{{user}}
I want to {{goal}}.
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.{{end}}
{{assistant}}
{{generate 'options' n=5 temperature=1.0 max_tokens=500}}{{end}}''')

discuss_best = guidance('''{{system}}
{{user}}
I want to {{goal}}.
Here is my plan:
{{best}}
Please elaborate on this plan, and tell me how to best accomplish it.{{end}}
{{assistant}}
{{generate 'plan' temperature=0.0 max_tokens=500}}''')

options = gen_options(system=system, user=user, assistant=assistant, end=end, goal=goal, eval=eval)
best = eval_idea(system=system, user=user, assistant=assistant, end=end, goal=goal, options=options['options'])
best_option = parse_best(options, best)
discuss_best(system=system, user=user, assistant=assistant, end=end, goal=goal, best=best_option)

{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to Make a lot of money.\nHere is my plan:\nInvest in high-growth stocks or industries and diversify your portfolio.\nPlease elaborate on this plan, and tell me how to best accomplish it.'}]}


<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to Make a lot of money.
Here is my plan:
Invest in high-growth stocks or industries and diversify your portfolio.
Please elaborate on this plan, and tell me how to best accomplish it.<|im_end|>
<|im_start|>assistant
To make a lot of money by investing in high-growth stocks or industries and diversifying your portfolio, follow these steps:

1. Research high-growth industries: Identify industries that are expected to grow significantly in the coming years. Some examples of high-growth industries include technology, renewable energy, healthcare, and e-commerce. Stay informed about market trends and news to spot potential growth opportunities.

2. Analyze individual stocks: Once you have identified high-growth industries, research individual companies within those industries. Look for companies with strong financials, innovative products or services, and a competitive edge in the market. Analyze their financial statements, growth rates, and market share to determine their potential for growth.

3. Diversify your portfolio: To minimize risk, it's essential to diversify your investments across different industries and asset classes. This means not putting all your money into one stock or industry. Instead, spread your investments across various sectors, such as technology, healthcare, finance, and consumer goods. You can also diversify by investing in different asset classes, such as stocks, bonds, and real estate.

4. Invest in low-cost index funds or ETFs: If you're not comfortable picking individual stocks, consider investing in low-cost index funds or exchange-traded funds (ETFs) that track the performance of high-growth industries or the overall market. These funds provide instant diversification and can help you gain exposure to multiple high-growth stocks with a single investment.

5. Dollar-cost averaging: To reduce the impact of market volatility, consider using a dollar-cost averaging strategy. This involves investing a fixed amount of money at regular intervals, regardless of market conditions. This approach can help you avoid making emotional decisions based on short-term market fluctuations and can lower the average cost per share over time.

6. Rebalance your portfolio: Periodically review and adjust your portfolio to ensure it remains aligned with your investment goals and risk tolerance. This may involve selling some investments and buying others to maintain the desired level of diversification.

7. Be patient and disciplined: Investing in high-growth stocks can be rewarding, but it requires patience and discipline. Be prepared to hold onto your investments for the long term, as short-term market fluctuations can be unpredictable. Stick to your investment plan and avoid making impulsive decisions based on market hype or fear.

8. Consult a financial advisor: If you're unsure about your investment strategy or need help managing your portfolio, consider consulting a financial advisor. They

In [77]:
def suggest_eval_discuss(goal):
    options = gen_options(system=system, user=user, assistant=assistant, end=end, goal=goal, eval=eval)
    best = eval_idea(system=system, user=user, assistant=assistant, end=end, goal=goal, options=options['options'])
    best_option = parse_best(options, best)
    discuss = discuss_best(system=system, user=user, assistant=assistant, end=end, goal=goal, best=best_option)
    return options, best, discuss
goal = 'Get rich'
a, b, c = suggest_eval_discuss(goal)
b

{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 1.0, 'top_p': 1.0, 'n': 5, 'stop': '{{end}}', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to Get rich.\nCan you please generate one option for how to accomplish this?\nPlease make the option very short, at most one line.'}]}
{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to Get rich.\nCan you please comment on the pros and cons of each of the following options, and then pick the best option?\n---\nOption 0: Invest in innovative, high-growth industries and consistently reinvest profits.\nOption 1: Invest in high growth potential stocks and hold for the long-term.\nOption 2: Invest in a high-demand, fast-growing industry with potential for significant returns.\nOption 3: Invest in high-growth stocks early and consist

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to Get rich.
Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---
Option 0: Invest in innovative, high-growth industries and consistently reinvest profits.
Option 1: Invest in high growth potential stocks and hold for the long-term.
Option 2: Invest in a high-demand, fast-growing industry with potential for significant returns.
Option 3: Invest in high-growth stocks early and consistently.
Option 4: Invest in high-potential stocks or cryptocurrencies with thorough research and a long-term strategy.
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.<|im_end|>
<|im_start|>assistant
Option 0: 
Pros: Diversified investment in multiple industries, compounding growth through reinvestment.
Cons: Requires constant monitoring and adjustment of portfolio.
 
Option 1: 
Pros: Long-term growth potential, less active management needed.
Cons: May miss out on short-term gains or market fluctuations.

Option 2: 
Pros: High potential for significant returns due to high demand.
Cons: Risk of industry saturation or sudden market changes.

Option 3: 
Pros: Early investment can lead to exponential growth.
Cons: Requires accurate prediction of future high-growth stocks.

Option 4: 
Pros: Thorough research can lead to informed investment decisions.
Cons: Cryptocurrencies can be highly volatile and unpredictable.

Best=1, as it provides long-term growth potential with less active management needed, allowing for a more stable and consistent investment strategy.

In [80]:
c

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to Get rich.
Here is my plan:
Invest in high growth potential stocks and hold for the long-term.
Please elaborate on this plan, and tell me how to best accomplish it.<|im_end|>
<|im_start|>assistant
Your plan to get rich by investing in high growth potential stocks and holding them for the long-term is a solid strategy. Here's an elaboration on your plan and some steps to help you accomplish it:

1. Research and educate yourself: Start by learning about the stock market, different types of stocks, and investment strategies. Read books, articles, and follow financial news to stay updated on market trends. You can also consider taking online courses or attending investment seminars to deepen your knowledge.

2. Set clear financial goals: Determine your financial objectives, such as the amount of money you want to make, the time horizon for achieving your goals, and your risk tolerance. This will help you make informed decisions about the stocks you choose to invest in.

3. Diversify your portfolio: Don't put all your eggs in one basket. Invest in a mix of high growth potential stocks across different sectors and industries. This will help spread the risk and increase the chances of earning higher returns.

4. Analyze stocks: Look for companies with strong financials, competitive advantages, and a history of growth. Analyze their financial statements, study their business models, and understand their growth prospects. You can use various financial ratios and metrics to evaluate a company's performance and potential for growth.

5. Invest consistently: Make a habit of investing a fixed amount of money regularly, regardless of market conditions. This approach, known as dollar-cost averaging, can help you avoid the pitfalls of market timing and reduce the impact of market volatility on your investments.

6. Be patient and disciplined: High growth potential stocks may take time to deliver significant returns. Stay committed to your long-term investment strategy and avoid making impulsive decisions based on short-term market fluctuations.

7. Monitor your investments: Regularly review your portfolio to ensure it remains aligned with your financial goals and risk tolerance. Rebalance your portfolio as needed by selling underperforming stocks and investing in new opportunities.

8. Consult a financial advisor: If you're unsure about your investment decisions or need guidance, consider consulting a professional financial advisor. They can help you develop a personalized investment strategy and provide expert advice on selecting high growth potential stocks.

By following these steps and staying committed to your long-term investment plan, you can increase your chances of getting rich through investing in high growth potential stocks. Remember, patience and discipline are key to achieving success in the stock market.

In [83]:
a, b, c = suggest_eval_discuss('convince my church to become elder-led congregationalist')
b

{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 1.0, 'top_p': 1.0, 'n': 5, 'stop': '{{end}}', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to convince my church to become elder-led congregationalist.\nCan you please generate one option for how to accomplish this?\nPlease make the option very short, at most one line.'}]}
{'model': 'gpt-4', 'max_tokens': 500, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'stop': '', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'I want to convince my church to become elder-led congregationalist.\nCan you please comment on the pros and cons of each of the following options, and then pick the best option?\n---\nOption 0: Present a well-researched proposal highlighting the benefits of elder-led congregationalism to the church leadership and members.\nOption 1: Propose a meeting to share research on the benefits of elder-led congregatio

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to convince my church to become elder-led congregationalist.
Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---
Option 0: Present a well-researched proposal highlighting the benefits of elder-led congregationalism to the church leadership and members.
Option 1: Propose a meeting to share research on the benefits of elder-led congregationalism, highlighting biblical support and successful examples.
Option 2: Present a well-researched proposal at a church meeting, highlighting the benefits of elder-led congregationalism for spiritual growth and community involvement.
Option 3: Present research on the benefits of elder-led congregationalism and share personal testimonies from churches that have successfully implemented this model.
Option 4: Present research on the benefits of elder-led congregationalism, focusing on spiritual growth and community engagement.
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.<|im_end|>
<|im_start|>assistant
Option 0:
Pros: Directly addresses church leadership and members with a well-researched proposal.
Cons: May not provide enough context or personal connection for some members.
---
Option 1:
Pros: Offers a dedicated meeting to discuss the topic and provide biblical support.
Cons: May not be as persuasive without personal testimonies or specific examples.
---
Option 2:
Pros: Addresses spiritual growth and community involvement, which are important aspects for many church members.
Cons: May not provide enough biblical support or personal testimonies.
---
Option 3:
Pros: Combines research with personal testimonies, making it more relatable and persuasive.
Cons: May not focus enough on spiritual growth and community engagement.
---
Option 4:
Pros: Focuses on spiritual growth and community engagement, which are important aspects for many church members.
Cons: May not provide enough biblical support or personal testimonies.
---
Best=3

In [84]:
c

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
I want to convince my church to become elder-led congregationalist.
Here is my plan:
Present research on the benefits of elder-led congregationalism and share personal testimonies from churches that have successfully implemented this model.
Please elaborate on this plan, and tell me how to best accomplish it.<|im_end|>
<|im_start|>assistant
1. Research and gather information: Begin by conducting thorough research on elder-led congregationalism. This should include understanding its biblical basis, historical background, and the benefits it offers to the church community. Collect data, articles, and books that support your argument and provide evidence of its effectiveness.

2. Identify key benefits: From your research, identify the key benefits of elder-led congregationalism that would be most relevant and appealing to your church community. Some possible benefits may include shared leadership, increased accountability, and a stronger focus on spiritual growth.

3. Collect personal testimonies: Reach out to churches that have successfully implemented an elder-led congregationalist model and ask for personal testimonies from their leaders and members. These testimonies can provide real-life examples of the positive impact this model has had on their church communities.

4. Prepare a presentation: Compile your research, key benefits, and personal testimonies into a well-organized and engaging presentation. Use visuals, such as slides or handouts, to help convey your message clearly and effectively. Be prepared to address any questions or concerns that may arise during your presentation.

5. Schedule a meeting with church leadership: Request a meeting with your church's leadership team to present your findings and discuss the possibility of transitioning to an elder-led congregationalist model. Be respectful of their time and come prepared with your presentation and any additional materials that may be helpful.

6. Engage the congregation: If the church leadership is open to the idea, organize a town hall-style meeting or a series of small group discussions to engage the congregation in the conversation. Share your research, key benefits, and personal testimonies, and encourage open dialogue and feedback from church members.

7. Address concerns and questions: Be prepared to address any concerns or questions that may arise during these discussions. Offer solutions and examples from other churches that have successfully made the transition to an elder-led congregationalist model.

8. Develop a transition plan: If the church leadership and congregation are supportive of the idea, work together to develop a detailed transition plan. This should include a timeline, clear steps for implementation, and a plan for training and equipping new elders.

9. Implement the transition: Once the plan has been approved, begin implementing the transition to an elder-led congregationalist model. This may involve selecting and training new elders, adjusting church bylaws, and communicating the changes to the congregation.

10. Monitor progress and make adjustments: As the transition progresses, regularly evaluate its effectiveness

In [81]:
guidance('''{{>b}}
----
{{>c}}''', b=b, c=c)

ValueError: not enough values to unpack (expected 2, got 1)